In [ ]:
import csv
import timeit
import numpy as np
import pandas as pd
import tensorflow as tf

#type these out as you explain what you are gonna do

In [ ]:
file = 'bank-additional-full.csv'

#show file

In [ ]:
pythonds=[line.split(';') for line in open(file).readlines()]

In [ ]:
pythonds[1]

In [ ]:
csvds = csv.reader(open(file),delimiter=';')

In [ ]:
csvds[1]

In [ ]:
pandasds = pd.read_csv(file,delimiter=';')

In [ ]:
pandasds.values

In [ ]:
#numpy op
npds = np.genfromtxt(file, delimiter=';', dtype=None,encoding="utf8")

In [ ]:
npds

In [ ]:
#timing ops
print(timeit.timeit(lambda: [line.split(';') for line in open('bank-additional-full.csv', 'r').readlines()], number = 10))
print(timeit.timeit(lambda: list(csv.reader(open('bank-additional-full.csv', 'r'), delimiter=';')), number = 10))
print(timeit.timeit(lambda: pd.read_csv('bank-additional-full.csv', delimiter=';'), number = 10))
print(timeit.timeit(lambda: np.genfromtxt('bank-additional-full.csv', delimiter=';', dtype=None,encoding="utf8"), number = 10))

In [ ]:
#prepare data for neural network

#label encoding
#one-hot encoding

In [ ]:
#in python
y = [0 if row[-1]=='"no"\n' else 1 for row in pythonds[1:]]
y

In [ ]:
npds[1:][:,-1]

In [ ]:
#in numpy
b,c = np.unique(npds[1:][:,-1], return_inverse=True)
print(b)
print(c)

In [ ]:
#in pandas 
pandasds["y"].astype('category')
pandasds["y"].cat.codes

In [ ]:
#timing ops
print(timeit.timeit(lambda: [0 if row[-1]=='"no"\n' else 1 for row in pythonds[1:]], number = 100))
print(timeit.timeit(lambda: np.unique(npds[1:][:,-1], return_inverse=True)[1], number = 100))
print(timeit.timeit(lambda: pandasds["y"].cat.codes, number = 100))

In [ ]:
#in pandas 
pd.get_dummies(pandasds, columns=["marital"]).head()

In [ ]:
#in numpy
b,c = np.unique(npds[1:][:,2], return_inverse=True)
np.eye(b.size)[c][:10]

In [ ]:
#in tensorflow
tf.one_hot(np.unique(npds[:,2], return_inverse=True)[1], b.size)

In [ ]:
#timing ops
print(timeit.timeit(lambda: pd.get_dummies(pandasds, columns=["marital",'job','education']), number = 100))
print(timeit.timeit(lambda: np.eye(b.size)[np.unique(npds[1:][:,2], return_inverse=True)[1]], number = 100))
print(timeit.timeit(lambda: tf.one_hot(np.unique(npds[1:][:,2], return_inverse=True)[1],b.size), number = 100))

In [ ]:
#data prep
df = pd.get_dummies(pandasds, columns=["job",'marital','education','default','housing','loan','month','day_of_week','poutcome'])
df["contact"] = df["contact"].astype('category')
df["contact"] = df["contact"].cat.codes
df = df.drop("duration", axis=1)
df["y"] = df["y"].astype('category')
df["y"] = df["y"].cat.codes

In [ ]:
df

In [ ]:
df['y'].value_counts()

In [ ]:
#retime upload to pandas df
print(timeit.timeit(lambda: pd.DataFrame([line.split(';') for line in open('bank-additional-full.csv', 'r').readlines()]), number = 10))
print(timeit.timeit(lambda: pd.DataFrame(list(csv.reader(open('bank-additional-full.csv', 'r'), delimiter=';'))), number = 10))
print(timeit.timeit(lambda: pd.read_csv('bank-additional-full.csv', delimiter=';'), number = 10))
print(timeit.timeit(lambda: pd.DataFrame(np.genfromtxt('bank-additional-full.csv', delimiter=';', dtype=None,encoding="utf8"), number = 10))

In [ ]:
#build, train model
df = df.sample(frac=1).reset_index(drop=True)
dftrain = df.iloc[:33000]
dftest = df.iloc[33000:]
dftrainy = dftrain['y']
dftesty = dftest['y']
dftrain = dftrain.drop("y", axis=1)
dftest = dftest.drop("y", axis=1)
xtrain = dftrain.values.astype(float)
ytrain = dftrainy.values.astype(float)
xtest = dftest.values.astype(float)
ytest = dftesty.values.astype(float)

model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(32, input_dim=61, kernel_initializer='random_normal'),
  tf.keras.layers.Dense(16, activation='relu',kernel_initializer='random_normal'),
  tf.keras.layers.Dense(16, activation='relu',kernel_initializer='random_normal'),
  tf.keras.layers.Dense(4, activation='relu',kernel_initializer='random_normal'),
  tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer='random_normal')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.AUC(),tf.keras.metrics.Recall(),'accuracy'])

model.fit(xtrain, ytrain, epochs=10)

model.evaluate(xtest, ytest)
preds = model.predict_classes(xtest, verbose=1)